# Python Course - Tutorial 10

### Exercise 1: Tidy Data

#### Overview: Wickham's Tidy Data Framework
Hadley Wickham's seminal paper on **Tidy Data** focuses on structuring datasets to facilitate analysis. The principles are:
1. **Each variable forms a column.**
2. **Each observation forms a row.**
3. **Each type of observational unit forms a table.**

Tidy datasets ensure a consistent structure, simplifying manipulation, visualization, and modeling. However, real-world datasets often deviate from this structure, requiring transformation.

#### Common Problems in Untidy Data
Wickham identifies four main types of untidy data:
1. **Column headers are values, not variable names.**
2. **Multiple variables are stored in one column.**
3. **Variables are stored in both rows and columns.**
4. **Multiple types of observational units are stored in the same table.**

In [2]:
# Importing pandas
import pandas as pd

### Exercise 1.1: Column Headers Are Values, Not Variable Names - Part 1

In the Pew dataset column headers represent income brackets rather than variable names. Our goal is to restructure the data into a tidy format, with columns for `religion`, `income`, and `frequency`.

#### Tasks:
1. Load the Pew dataset from a CSV file.
2. Inspect the data to identify untidy elements.
3. Use Pandas' `melt` function to:
   - Transform the income brackets (column headers) into a new column named `income`.
   - Ensure the original `religion` column remains intact.
   - Create a new column named `frequency` to hold the corresponding values.
4. Sort the resulting tidy dataset by `religion` (alphabetically) and `income` (numerically).


#### Dataset Preview:

The untidy data might look like this:

| religion           | <$10k | $10–20k | $20–30k | $30–40k | $40–50k | $50–75k |
|--------------------|--------|---------|---------|---------|---------|---------|
| Agnostic           | 27     | 34      | 60      | 81      | 76      | 137     |
| Atheist            | 12     | 27      | 37      | 52      | 35      | 70      |
| Buddhist           | 27     | 21      | 30      | 34      | 33      | 58      |
| Catholic           | 418    | 617     | 732     | 670     | 638     | 1116    |
| Don’t know/refused | 15     | 14      | 15      | 11      | 10      | 35      |

Using `pd.melt`, we can achieve the following tidy dataset:

| religion           | income    | frequency |
|--------------------|-----------|-----------|
| Agnostic           | <$10k     | 27        |
| Agnostic           | $10–20k   | 34        |
| Agnostic           | $20–30k   | 60        |
| Atheist            | <$10k     | 12        |
| Atheist            | $10–20k   | 27        |
| Buddhist           | <$10k     | 27        |



In [ ]:
# Import the Pew sample dataset
pew_data = pd.read_csv("data/pew_sample.csv")

# Reshape the Pew dataset from wide format to long format
pew_long_format = pd.melt(
    frame=pew_data,
    id_vars=["religion"],  # Keep the "religion" column fixed
    value_vars=pew_data.columns[1:],  # Use all other columns as value variables
    var_name="income_bracket",  
    value_name="frequency",  
)

# Sort the reshaped data by religion and income_bracket
pew_long_sorted = pew_long_format.sort_values(by=["religion", "income_bracket"])

# Display the first rows of sorted data
pew_long_sorted.head()

### Exercise 1.1: Column Headers Are Values, Not Variable Names - Part 2

The **Billboard dataset** is stored in a wide format. The data contains weekly rankings (`wk1`, `wk2`, ..., `wk75`) spread across columns. Your task is to transform it into a **long format** where each row corresponds to a specific song's ranking in a particular week.

#### Tasks:
1. **Inspect the Dataset**: Familiarize yourself with the structure of the Billboard dataset. Identify columns to retain (e.g., `year`, `artist`, `track`, `time`, `date.entered`) and those to melt (e.g., `wk1`, `wk2`, ..., `wk75`).
2. **Use `pd.melt`**:
   - Reshape the dataset so that all weekly ranking columns (`wk1`, `wk2`, ..., `wk75`) are combined into a single `week` column.
   - Create a new column `rank` to hold the corresponding ranking values.
3. **Clean Up the Week Column**:
   - Use string methods to extract only the numeric part of the week values (e.g., `wk1` → `1`).
   - Convert the `week` column into an integer type for easier manipulation.

**Hint:** Apply the `.str.replace()` or `.str.extract()` methods to clean up the `week` column.

#### Dataset Preview:
**Original Untidy Data (Wide Format)**:

| year | artist       | track                    | time | date.entered | wk1 | wk2 | wk3 | ... | wk75 |
|------|--------------|--------------------------|------|--------------|-----|-----|-----|-----|------|
| 2000 | 2 Pac        | Baby Don’t Cry           | 4:22 | 2000-02-26   | 87  | 82  | 72  | ... | NaN  |
| 2000 | 2Ge+her      | The Hardest Part Of ...  | 3:15 | 2000-09-02   | 91  | 87  | 82  | ... | NaN  |
| 2000 | 3 Doors Down | Kryptonite               | 3:53 | 2000-04-08   | 81  | 70  | 66  | ... | NaN  |


**After Transformation (Long Format)**:

| year | artist       | time | track                  | date       | week | rank |
|------|--------------|------|------------------------|------------|------|------|
| 2000 | 2 Pac        | 4:22 | Baby Don’t Cry         | 2000-02-26 | 1    | 87   |
| 2000 | 2 Pac        | 4:22 | Baby Don’t Cry         | 2000-03-04 | 2    | 82   |
| 2000 | 2 Pac        | 4:22 | Baby Don’t Cry         | 2000-03-11 | 3    | 72   |
| 2000 | 2Ge+her      | 3:15 | The Hardest Part Of ...| 2000-09-02 | 1    | 91   |
| 2000 | 2Ge+her      | 3:15 | The Hardest Part Of ...| 2000-09-09 | 2    | 87   |
| 2000 | 2Ge+her      | 3:15 | The Hardest Part Of ...| 2000-09-02 | 3    | 92   |
| 2000 | 3 Doors Down | 3:53 | Kryptonite             | 2000-04-08 | 1    | 81   |
| 2000 | 3 Doors Down | 3:53 | Kryptonite             | 2000-04-15 | 2    | 70   |
| 2000 | 3 Doors Down | 3:53 | Kryptonite             | 2000-04-22 | 3    | 66   |



In [ ]:
# Import the Billboard dataset
billboard_data = pd.read_csv("data/billboard.csv")

# Define columns to keep fixed during the transformation
id_columns = [
    "year",
    "artist.inverted",
    "track",
    "time",
    "genre",
    "date.entered",
    "date.peaked",
]

# Reshape the dataset from wide to long format
# Columns not in id_columns will become the 'week' column in the resulting DataFrame
billboard_long_format = pd.melt(
    frame=billboard_data,
    id_vars=id_columns,
    var_name="week",  # Name for the variable column (week number column)
    value_name="rank",  # Name for the value column (chart rank)
)

# Clean the 'week' column by removing unwanted text patterns and convert to integer
billboard_long_format["week"] = billboard_long_format["week"].str.replace(
    "x|st.week|th.week|nd.week|rd.week", "", regex=True
).astype(int)

# Calculate the actual chart date by adding the week offset to the 'date.entered'
billboard_long_format["date"] = pd.to_datetime(billboard_long_format["date.entered"]) + pd.to_timedelta(
    (billboard_long_format["week"] - 1) * 7, "d"
)

# Sort the dataset by track and date for better readability
billboard_long_sorted = billboard_long_format.sort_values(by=["track", "date"])

# Drop rows with missing values (e.g., ranks not recorded for certain weeks)
billboard_cleaned = billboard_long_sorted.dropna()

# Display the cleaned and processed DataFrame
billboard_cleaned

### Exercise 1.2: Multiple Variables Are Stored in One Column

In the tuberculosis (TB) dataset multiple variables (e.g., sex and age group) are combined into a single column. Your goal is to separate these into distinct columns, making the dataset tidy.


#### Tasks:
1. **Reshape the Dataset**: Use `pd.melt` to convert the wide format into a long format where the demographic variables (e.g., `m014`, `f1524`) are stored in a single column.
2. **Split the Combined Column**: Extract the `sex` and `age` variables from the demographic column.
3. **Clean the Age Values**: Use the `map` method with a dictionary to convert the age codes (e.g., `014`, `1524`) into human-readable ranges (e.g., `0–14`, `15–24`).


#### Dataset Preview:
**Original Untidy Data**:

| country | year | m014 | m1524 | m2534 | m3544 | m4554 | m5564 | m65 | mu  | f014 | f1524 | f2534 |
|---------|------|------|-------|-------|-------|-------|-------|-----|-----|------|-------|-------|
| AD      | 2000 | 0    | 0     | 1     | 0     | 0     | 0     | 0   | --- | ---  | ---   | ---   |
| AE      | 2000 | 2    | 4     | 4     | 6     | 5     | 12    | 10  | --- | 3    | 6     | 5     |
| AF      | 2000 | 52   | 228   | 183   | 149   | 129   | 94    | 80  | --- | 93   | 142   | 128   |

**After Transformation (Tidy Format)**:

| country | year | sex | age  | cases |
|---------|------|-----|------|-------|
| AD      | 2000 | m   | 0–14 | 0     |
| AE      | 2000 | m   | 0–14 | 2     |
| AF      | 2000 | m   | 0–14 | 52    |
| AD      | 2000 | f   | 15–24| 0     |
| AE      | 2000 | f   | 15–24| 6     |
| AF      | 2000 | f   | 15–24| 142   |


In [ ]:
# Import the TB dataset
tb_data = pd.read_csv("data/tb_sample.csv")

# Reshape the dataset from wide to long format
tb_long_format = pd.melt(
    tb_data,
    id_vars=["country", "year"],  # Keep "country" and "year" as fixed identifiers
    value_vars=list(tb_data.columns)[2:],  # Columns to unpivot
    var_name="column",  
    value_name="cases",  
)

# Extract the 'sex' information from the 'column' field
tb_long_format["sex"] = tb_long_format["column"].str[0]

# Extract the 'age' information and map it to readable age groups
tb_long_format["age"] = tb_long_format["column"].str[1:].map(
    {
        "014": "0-14",
        "1524": "15-24",
        "2534": "25-34",
        "3544": "35-44",
        "4554": "45-54",
        "5564": "55-64",
        "65": "65+",
    }
)

# Drop unnecessary columns after extracting useful information
tb_cleaned = tb_long_format.drop(columns=["column"])

# Display the cleaned and formatted tb dataset
tb_cleaned.head()

### Exercise 1.3: Variables Are Stored in Both Rows and Columns (Bonus!)

The weather dataset contains variables such as minimum and maximum temperature (`tmin`, `tmax`) that stored across both rows and columns. Your task is to tidy the dataset by ensuring each variable is represented in its own column.



#### Tasks:
1. **Reshape the Dataset**:
   - Use `pd.melt` to gather all day columns (`d1`, `d2`, ..., `d31`) into a single column, converting the wide format into a long format.
   - Create a new `date` column by combining `year`, `month`, and the day extracted from the melted column.

2. **Separate Variables**:
   - Pivot the data so that `tmin` and `tmax` are stored as separate columns, with each row representing a unique `date`.

3. **Handle Missing Values**:
   - Remove rows where the `value` is missing (`—`).

**Hints:**
- Combine `year`, `month`, and `day` into a single `date` column using `pd.to_datetime` or string formatting.
- Use `pivot` or `unstack` to move the `element` values (`tmin`, `tmax`) into separate columns.

#### Dataset Preview
**Original Untidy Data**:

| id      | year | month | element | d1   | d2   | d3   | ... | d8  |
|---------|------|-------|---------|------|------|------|-----|------|
| MX17004 | 2010 | 1     | tmax    | —    | —    | —    | ... | —    |
| MX17004 | 2010 | 1     | tmin    | —    | —    | —    | ... | —    |
| MX17004 | 2010 | 2     | tmax    | —    | 27.3 | 24.1 | ... | —    |
| MX17004 | 2010 | 2     | tmin    | —    | 14.4 | 14.4 | ... | —    |

**After Transformation (Tidy Format)**:

| id      | date       | tmax  | tmin  |
|---------|------------|-------|-------|
| MX17004 | 2010-02-02 | 27.3  | 14.4  |
| MX17004 | 2010-02-03 | 24.1  | 14.4  |
| MX17004 | 2010-03-05 | 32.1  | 14.2  |

In [ ]:
# Import the weather sample dataset
weather_data = pd.read_csv("data/weather_sample.csv")

# Transform the data from wide format to long format
weather_long = pd.melt(
    frame=weather_data,
    id_vars=["id", "year", "month", "element"],
    var_name="day",
    value_name="value",
)

# Remove the leading "d" from the day column and convert to integer
weather_long["day"] = weather_long["day"].str[1:].astype("int")

# Combine year, month, and day into a single date column in YYYY-MM-DD format
weather_long["date"] = weather_long[["year", "month", "day"]].apply(
    lambda row: "{:4d}-{:02d}-{:02d}".format(*row), axis=1
)

# Filter out rows with missing values and keep relevant columns
weather_filtered = weather_long.loc[
    ~weather_long["value"].isna(), ["id", "date", "element", "value"]
]

# Set index to a multi-index of id, date, and element
weather_indexed = weather_filtered.set_index(["id", "date", "element"])

# Unstack the element level of the index to create separate columns for each element
weather_wide = weather_indexed.unstack()

# Flatten the multi-index column names
weather_wide.columns = list(weather_wide.columns.get_level_values("element"))

# Reset the index to convert it back into a DataFrame
weather_cleaned = weather_wide.reset_index()

# Display the formatted dataframe
weather_cleaned

### Exercise 1.4: Multiple Types of Observational Units Are Stored in the Same Table

The Billboard dataset, contains information about tracks (e.g., artist, track title, and time) as well as their rankings over time. These represent two distinct types of observational units. Your goal is to split the dataset into two separate tables:
1. One table for track information.
2. Another table for weekly rankings.


#### Steps to Complete:

1. **Extract Track Information**: Identify the unique combinations of `artist`, `track`, and `time` and assign each unique combination a unique `id`.
2. **Separate Weekly Rankings**: Create a new table for rankings, using the `id` from the track table to link the two datasets.
3. **Join Tables When Needed**: Use the `merge` function to link track information with ranking data when required for analysis.

**Hint:** Use `pd.drop_duplicates` to extract unique rows for the track table.


#### Dataset Preview
**Original Untidy Data** (Mixed Observational Units):

| year | artist          | track              | time | date       | week | rank |
|------|-----------------|--------------------|------|------------|------|------|
| 2000 | Nelly           | Country Grammar    | 4:17 | 2000-04-29 | 1    | 100  |
| 2000 | Nelly           | Country Grammar    | 4:17 | 2000-05-06 | 2    | 99   |
| 2000 | Nelly           | Country Grammar    | 4:17 | 2000-05-13 | 3    | 96   |
| 2000 | Nelly           | Country Grammar    | 4:17 | 2000-05-20 | 4    | 76   |
| 2000 | Nelly           | Country Grammar    | 4:17 | 2000-05-27 | 5    | 55   |


**After Transformation: Two Separate Tables**

1. **Track Table** (One row per unique track):

| id | artist          | track              | time |
|----|-----------------|--------------------|------|
| 1  | Nelly           | Country Grammar    | 4:17 |

2. **Rank Table** (One row per weekly ranking):

| id | date       | rank |
|----|------------|------|
| 1  | 2000-04-29 | 100  |
| 1  | 2000-05-06 | 99   |
| 1  | 2000-05-13 | 96   |
| 1  | 2000-05-20 | 76   |
| 1  | 2000-05-27 | 55   |

In [ ]:
# Import the billboard DataFrame from exercise 1.2 and ensure it is in long format.

# Extract unique tracks along with their artist and duration
unique_tracks = billboard_cleaned[["artist.inverted", "track", "time"]].drop_duplicates()

# Assign a unique ID to each track
unique_tracks.insert(0, "track_id", range(1, len(unique_tracks) + 1))

# Merge the original DataFrame with the unique tracks, adding track IDs to each record
billboard_with_track_id = pd.merge(billboard_cleaned, unique_tracks, on=["artist.inverted", "track", "time"])

# Retain only the relevant columns: track ID, date, and rank
billboard_with_track_id = billboard_with_track_id[["track_id", "date", "rank"]]

# Merge data for a specific artist (e.g., "Nelly") with track details
pd.merge(
    unique_tracks[unique_tracks["artist.inverted"] == "Nelly"],
    billboard_with_track_id,
    on=["track_id"]
)

### Exercise 2: Visualizing the Palmer Penguins Dataset with Seaborn

The Palmer Penguins dataset provides information about three penguin species—Adélie, Chinstrap, and Gentoo—observed in Antarctica. This dataset contains details such as body mass, flipper length, and bill dimensions, making it ideal for data visualization practice.

#### Dataset Overview
The dataset includes the following key columns:
- `species`: The penguin species (Adélie, Chinstrap, Gentoo).
- `island`: The island where the penguins were observed.
- `bill_length_mm`: Length of the penguin's bill (in millimeters).
- `bill_depth_mm`: Depth of the penguin's bill (in millimeters).
- `flipper_length_mm`: Length of the penguin's flipper (in millimeters).
- `body_mass_g`: Body mass of the penguin (in grams).
- `sex`: The penguin's sex (male, female).

The dataset can be loaded directly from the seaborn repository using the URL with `pd.read_csv(url)`:  
`https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv`

In the previous tutorial, we briefly explored how to create plots using **matplotlib**. Today, we will focus on **seaborn**, a powerful Python library built on top of matplotlib that allows us to create elegant and informative visualizations with minimal effort. By using seaborn, we can generate complex and aesthetically pleasing plots with just a few lines of code.

#### Tasks

1. **Visualize the Distribution of Flipper Length**:
   - Create a histogram or kernel density plot (KDE) of `flipper_length_mm`, with the distributions differentiated by `species` using the `hue` parameter.

2. **Explore the Relationship Between Body Mass and Flipper Length**:
   - Use a scatter plot to visualize the relationship between `body_mass_g` and `flipper_length_mm`.
   - Differentiate the points by `species` using the `hue` parameter and use distinct markers for each species.

3. **Compare Body Mass Across Species**:
   - Create a boxplot to compare the distributions of `body_mass_g` across the three penguin species.

4. **Analyze Pairwise Relationships**:
   - Generate a pair plot for the numerical variables (`bill_length_mm`, `bill_depth_mm`, `flipper_length_mm`, `body_mass_g`), with the points colored by `species`. Use a KDE plot for the diagonal.


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the Palmer Penguins dataset
url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv"
penguins = pd.read_csv(url)

# Display the first few rows of the dataset
display(penguins.head())

# 1. Distribution of Flipper Length
sns.histplot(
    data=penguins, 
    x="flipper_length_mm", 
    hue="species", 
    kde=True, 
    palette="pastel",
    bins=20
)
plt.title("Distribution of Flipper Length by Species")
plt.xlabel("Flipper Length (mm)")
plt.ylabel("Density")
plt.show()

# 2. Relationship Between Body Mass and Flipper Length
sns.scatterplot(
    data=penguins, 
    x="flipper_length_mm", 
    y="body_mass_g", 
    hue="species", 
    style="species", 
    palette="Set2"
)
plt.title("Body Mass vs. Flipper Length by Species")
plt.xlabel("Flipper Length (mm)")
plt.ylabel("Body Mass (g)")
plt.show()

# 3. Comparing Body Mass Across Species
sns.boxplot(
    data=penguins, 
    x="species", 
    y="body_mass_g", 
    palette="viridis", 
    hue="species"
)
plt.title("Body Mass Distribution Across Species")
plt.xlabel("Species")
plt.ylabel("Body Mass (g)")
plt.show()

# 4. Pair Plot of Numerical Variables
sns.pairplot(
    data=penguins, 
    hue="species", 
    palette="husl", 
    diag_kind="kde", 
    markers=["o", "s", "D"]
)
plt.show()

### Exercise 3: Analyzing Stock Correlations and Volatility with Yahoo Finance Data

With thousands of stocks being traded daily, making informed decisions about portfolio composition requires extracting meaningful insights from stock data. One useful strategy involves identifying **low-correlation stocks** to minimize portfolio risk, as combining such stocks helps diversify the portfolio and reduce its overall volatility.

In this exercise, you will:
1. **Download stock data** using the `yfinance` library for a list of tickers.
2. **Calculate daily returns and correlations** between stock pairs.
3. **Determine the two stocks with the lowest correlation** for potential use in hedging strategies.

#### Why Low-Correlation Stocks?
When combining two stocks with low or negative correlation:
- The **expected return** of the portfolio is unaffected by the correlation and is given by:
  
  $$
  \mu_p = w_1 \mu_1 + w_2 \mu_2
  $$
  
  where:
  - $ w_1, w_2 $: Portfolio weights for the two stocks.
  - $ \mu_1, \mu_2 $: Expected returns of the two stocks.

- The **volatility** of the portfolio, however, is reduced and depends on the correlation ($ \rho $):

  $$
  \sigma_p = \sqrt{w_1^2 \sigma_1^2 + w_2^2 \sigma_2^2 + 2 w_1 w_2 \sigma_1 \sigma_2 \rho}
  $$

  where:
  - $ \sigma_1, \sigma_2 $: Volatilities of the two stocks.
  - $ \rho $: Correlation between the two stocks.

This formula highlights that lowering the correlation ($ \rho $) reduces the portfolio's overall volatility ($ \sigma_p $), making it an essential consideration for diversification and risk reduction.



#### Tasks
1. **Read the list of stock tickers**:
   - Read a text file (`sp500-ticker-list.txt`) containing a list of stock tickers.

2. **Download stock data**:
   - Use the `yfinance` library to fetch historical data from 2000 and 2024 for each stock.
   - Save the data to individual CSV files in a `stock_data/` directory.

3. **Calculate daily returns**:
   - Compute the daily returns as the percentage change of adjusted closing prices for each stock.

4. **Calculate the correlation matrix**:
   - Use NumPy to compute the correlation matrix of the daily returns.

5. **Find the two least correlated stocks**:
   - Identify the two stocks with the lowest correlation from the matrix.


In [ ]:
import pathlib
import yfinance as yf
import numpy as np
import pandas as pd
import glob

# Step 1: Read the list of stock tickers from a file
with open("data/sp500-ticker-list.txt") as f:
    stock_list = f.read().splitlines()

# Step 2: Create a directory for the data
pathlib.Path("stock_data").mkdir(exist_ok=True)

# Step 3: Download stock data and save to CSV files
for stock in stock_list:
    data = yf.download(stock, start="2000-01-01", end="2024-12-31", interval="1d")
    data.columns = data.columns.get_level_values(0)
    file_name = f"stock_data/{stock}.csv"
    data.to_csv(file_name)
    # print(f"{stock} data saved to {file_name}")

# Step 4: Load stock data and calculate daily returns
missing_data = []
stock_data = {}
for file in glob.glob("stock_data/*.csv"):
    stock_name = pathlib.Path(file).stem
    df = pd.read_csv(file)
    try:
        df["Daily_Return"] = df["Close"].pct_change()  # Calculate daily returns
    except:
        missing_data.append(stock_name) # Append to missing_data list, if the df is empty
        continue
    df.dropna(subset=["Daily_Return"], inplace=True)  # Drop rows with NaN daily returns
    stock_data[stock_name] = df["Daily_Return"].values

# Step 5: Filter stocks with complete data
max_data_len = max(len(data) for data in stock_data.values())
stock_data = {name: data for name, data in stock_data.items() if len(data) == max_data_len}

# Step 6: Compute the correlation matrix
correlation_matrix = np.corrcoef(list(stock_data.values()))

# Step 7: Find the two stocks with the smallest correlation
i, j = np.unravel_index(
    np.argmin(correlation_matrix),
    correlation_matrix.shape
)
stock_names = list(stock_data.keys())
stock1, stock2 = stock_names[i], stock_names[j]

print(f"The two stocks with the lowest correlation are: {stock1} and {stock2}")
print(f"Excluded stocks (due to missing data) are: {missing_data}")
